In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

2023-12-04 22:14:17.494956: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-04 22:14:17.495056: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-04 22:14:17.519570: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-04 22:14:17.579026: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-04 22:14:18.702609: W tensorflow/compiler/tf2

In [69]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0)
plt.rcParams['image.cmap'] = 'Greys'

print(tf.__version__)
print(tf.config.get_visible_devices())
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

2.15.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [70]:
(ds_train, ds_test) = tfds.load('mnist', split=['train', 'test'], shuffle_files=True, as_supervised=True)
print(ds_train)

<_PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>


In [71]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(normalize_img)
ds_train = ds_train.batch(100)

ds_test_norm = ds_test.map(normalize_img)
ds_test = ds_test_norm.batch(100)

(ds_train, ds_test)

(<_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

In [72]:
# wx + b
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(10),
  tf.keras.layers.Softmax()
])

# loss fn
cross_en = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.SUM)

model.compile(loss=cross_en, metrics=['accuracy'])

In [73]:
checkpoint = "training-simple-mnist/1000.ckpt"

try:
    model.load_weights(checkpoint)
except Exception as e:
    print(e)
    model.fit(ds_train, epochs=1000)
    model.save_weights(checkpoint)

print(model.summary())
print(model.get_config())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                7850      
                                                                 
 softmax_2 (Softmax)         (None, 10)                0         
                                                                 
Total params: 7850 (30.66 KB)
Trainable params: 7850 (30.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
{'name': 'sequential_2', 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_input_shape': (None, 28, 28), 'dtype': 'float32', 'sparse': False, 'ragged': False, 'name': 'flatten_2_input'}, 'registered_name': None}, {'module': 'keras.layers', 'clas

In [77]:
# test 1
for d in ds_test.take(1):
    img, label = d
    img_, label_ = normalize_img(img, label)
    # img_ = np.reshape(img_, (1, 28, 28))
    out = model.predict(img_)
    print(out)
    print("predicted: ", out.argmax(axis=0), f"({out.max()})")
    print("actual: ", label_.numpy())

4/4 [==============================] - 0s 927us/step
[[0.00178988 0.12952857 0.05930356 0.01328077 0.04860185 0.62771136
  0.00342846 0.09864821 0.00262953 0.01507791]
 [0.00220434 0.11356433 0.05972067 0.01206429 0.04779417 0.6457465
  0.00398579 0.0982242  0.00295765 0.01373812]
 [0.0017458  0.11850025 0.05440576 0.01304988 0.05354543 0.6133646
  0.00352935 0.12242266 0.002714   0.01672228]
 [0.00177847 0.1470602  0.05517699 0.01248505 0.04655268 0.6056687
  0.00351205 0.10938215 0.00271061 0.01567311]
 [0.00176678 0.12640393 0.05482807 0.01299749 0.04863904 0.6055508
  0.00331347 0.12744038 0.00258864 0.01647137]
 [0.0017769  0.13068055 0.05700647 0.01255093 0.05005436 0.62328464
  0.00400692 0.1030016  0.00268008 0.01495748]
 [0.00205562 0.11149572 0.05621679 0.01227714 0.04647167 0.6389664
  0.00329309 0.11094745 0.00275246 0.01552371]
 [0.00177824 0.14011261 0.05573915 0.01292163 0.04884734 0.6209904
  0.0037301  0.09837438 0.00269171 0.01481437]
 [0.00177177 0.13289863 0.0572401

2023-12-04 23:56:28.467932: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [67]:
# overall test
test_loss, test_acc = model.evaluate(ds_test, verbose=2)
print('\nTest accuracy:', test_acc)

100/100 - 0s - 272ms/epoch - 3ms/step


TypeError: argument of type 'NoneType' is not iterable